In [2]:
import numpy as np
import pandas as pd
# from matplotlib import pyplot as plt
# import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import cross_val_score,train_test_split
# from sklearn.linear_model import LogisticRegression,SGDClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB
# from xgboost import XGBClassifier
# from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
import tensorflow as tf

In [3]:
breast_cancer_df=pd.read_csv("cancer_detection.csv")
breast_cancer_df=breast_cancer_df.drop(labels={
    "Unnamed: 32",
    "id"},axis=1)
print(breast_cancer_df.head())

  diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0         M        17.99         10.38          122.80     1001.0   
1         M        20.57         17.77          132.90     1326.0   
2         M        19.69         21.25          130.00     1203.0   
3         M        11.42         20.38           77.58      386.1   
4         M        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   symmetry_mean  ...  radius_worst  texture_worst  perimeter_worst  \
0         0.2419  ...         25.38          17.33 

In [4]:
le=LabelEncoder()
breast_cancer_df["diagnosis"]=le.fit_transform(breast_cancer_df["diagnosis"])
print(breast_cancer_df.head())
print(breast_cancer_df["diagnosis"].value_counts())

   diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0          1        17.99         10.38          122.80     1001.0   
1          1        20.57         17.77          132.90     1326.0   
2          1        19.69         21.25          130.00     1203.0   
3          1        11.42         20.38           77.58      386.1   
4          1        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   symmetry_mean  ...  radius_worst  texture_worst  perimeter_worst  \
0         0.2419  ...         25.38          

In [5]:
# print(breast_cancer_df.columns)
correlation_matrix=breast_cancer_df.corr()
# plt.figure(figsize=(20,20))
# sns.heatmap(correlation_matrix,annot=True)
correlation_coefficient=correlation_matrix.loc["diagnosis"]
print(correlation_coefficient.dtype)
# print(temp_correlation_coefficient[1])
# print(correlation_coefficient[correlation_coefficient>=0.5])

float64


In [6]:
new_correlation_coefficient=abs(correlation_coefficient)
temp_correlation_coefficient=new_correlation_coefficient.sort_values(ascending=False)
top=temp_correlation_coefficient[1:16]
bottom=temp_correlation_coefficient[16:]
# print(top)
print("\n")
top_correlation_coefficient=correlation_coefficient[top.index]
print(top_correlation_coefficient)



concave points_worst    0.793566
perimeter_worst         0.782914
concave points_mean     0.776614
radius_worst            0.776454
perimeter_mean          0.742636
area_worst              0.733825
radius_mean             0.730029
area_mean               0.708984
concavity_mean          0.696360
concavity_worst         0.659610
compactness_mean        0.596534
compactness_worst       0.590998
radius_se               0.567134
perimeter_se            0.556141
area_se                 0.548236
Name: diagnosis, dtype: float64


In [7]:
new_breast_cancer_df=breast_cancer_df.drop(labels=bottom.index,axis=1)
print(new_breast_cancer_df.head())

   diagnosis  radius_mean  perimeter_mean  area_mean  compactness_mean  \
0          1        17.99          122.80     1001.0           0.27760   
1          1        20.57          132.90     1326.0           0.07864   
2          1        19.69          130.00     1203.0           0.15990   
3          1        11.42           77.58      386.1           0.28390   
4          1        20.29          135.10     1297.0           0.13280   

   concavity_mean  concave points_mean  radius_se  perimeter_se  area_se  \
0          0.3001              0.14710     1.0950         8.589   153.40   
1          0.0869              0.07017     0.5435         3.398    74.08   
2          0.1974              0.12790     0.7456         4.585    94.03   
3          0.2414              0.10520     0.4956         3.445    27.23   
4          0.1980              0.10430     0.7572         5.438    94.44   

   radius_worst  perimeter_worst  area_worst  compactness_worst  \
0         25.38           184.6

In [8]:
scaler=StandardScaler()
diagnosis_df=new_breast_cancer_df["diagnosis"]
scaled_breast_cancer_array=scaler.fit_transform(new_breast_cancer_df[top.index])
print(scaled_breast_cancer_array.shape)
scaled_breast_cancer_inputs=pd.DataFrame(scaled_breast_cancer_array,columns=top.index)
scaled_breast_cancer_df=scaled_breast_cancer_inputs.join(diagnosis_df)
print(scaled_breast_cancer_df.head(),"\n",scaled_breast_cancer_df["diagnosis"].value_counts())
for col in scaled_breast_cancer_df.columns:
    print(scaled_breast_cancer_df[col].describe())

(569, 15)
   concave points_worst  perimeter_worst  concave points_mean  radius_worst  \
0              2.296076         2.303601             2.532475      1.886690   
1              1.087084         1.535126             0.548144      1.805927   
2              1.955000         1.347475             2.037231      1.511870   
3              2.175786        -0.249939             1.451707     -0.281464   
4              0.729259         1.338539             1.428493      1.298575   

   perimeter_mean  area_worst  radius_mean  area_mean  concavity_mean  \
0        1.269934    2.001237     1.097064   0.984375        2.652874   
1        1.685955    1.890489     1.829821   1.908708       -0.023846   
2        1.566503    1.456285     1.579888   1.558884        1.363478   
3       -0.592687   -0.550021    -0.768909  -0.764464        1.915897   
4        1.776573    1.220724     1.750297   1.826229        1.371011   

   concavity_worst  compactness_mean  compactness_worst  radius_se  \
0     

In [9]:
breast_cancer_train,breast_cancer_test=train_test_split(scaled_breast_cancer_df,test_size=0.2,random_state=42,shuffle=True,stratify=scaled_breast_cancer_df["diagnosis"])

In [10]:
breast_cancer_train.diagnosis.value_counts()
print(breast_cancer_train.shape)

(455, 16)


In [11]:
breast_cancer_test.diagnosis.value_counts()

diagnosis
0    72
1    42
Name: count, dtype: int64

### Neural Network

In [12]:
import keras
model=keras.Sequential([
    keras.layers.Dense(16,activation="relu",input_shape=(15,)),
    keras.layers.Dense(8,activation="relu"),
    keras.layers.Dense(1,activation="sigmoid")
])

c:\Users\Areeb\Desktop\VS CODE Workspace\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [14]:
model.fit(breast_cancer_train[top.index],breast_cancer_train["diagnosis"],epochs=100)

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6248 - loss: 0.8456  
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6753 - loss: 0.6032
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8174 - loss: 0.4505
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9061 - loss: 0.3537
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9153 - loss: 0.3088
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9339 - loss: 0.2622
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9140 - loss: 0.2542
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9528 - loss: 0.2115
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9262 - loss: 0.2159
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9310 - loss: 0.2166
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 146us/step - accuracy: 0.9321 - loss: 0.1913
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accur

In [15]:
loss,accuracy=model.evaluate(breast_cancer_test[top.index],breast_cancer_test["diagnosis"])
print(f"the accuracy is {accuracy*100}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9613 - loss: 0.0666 
the accuracy is 97.36841917037964


In [16]:
import pickle

In [17]:
pickle.dump(scaler,open("scaler.pkl","wb"))
pickle.dump(model,open("model.pkl","wb"))

In [18]:
print(model.predict(np.array([2.296076,2.303601,2.532475,1.886690,2.303601,2.001237,1.097064,0.984375,2.652874,2.109526,3.283515,2333.468,2.489734,2.833031,2.487578]).reshape(1, -1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[[1.]]


In [19]:
print(top.index)

Index(['concave points_worst', 'perimeter_worst', 'concave points_mean',
       'radius_worst', 'perimeter_mean', 'area_worst', 'radius_mean',
       'area_mean', 'concavity_mean', 'concavity_worst', 'compactness_mean',
       'compactness_worst', 'radius_se', 'perimeter_se', 'area_se'],
      dtype='object')
